## Variables y mutabilidad 

Por defecto las variables en Rust son **inmutables**. Este es uno de los muchos codazos que brinda Rust para escribir código en una manera que aproveche la seguridad y facilidad de hacer concurrencia que ofrece Rust.

Cuando una variable es inmutable, una vez que un valor está vinculado a un nombre, no puede cambiar ese valor. Para ilustrar esto, veremos un ejemplo: 

In [16]:
fn main() {
    let x = 5;
    format!("The value of x is: {}", x);
    x = 6;
    eprintln!("The value of x is: {}", x);
}
main()

Error: cannot assign twice to immutable variable `x`

El mensaje de error indica que la causa del error es que no puede asignar dos veces a la variable inmutable x, porque trató de asignar un segundo valor a la variable inmutable x.

Los enlaces variables son inmutables por defecto, pero esto se puede anular utilizando el modificador **mut.**

In [30]:
fn main() {
    let _immutable_binding = 1;
    let mut mutable_binding = 1;

    println!("Before mutation: {}", mutable_binding);

    // Ok
    mutable_binding += 1;

    println!("After mutation: {}", mutable_binding);

    // Error!
    _immutable_binding += 1;
    // FIXME ^ Comment out this line
}
main()

Error: cannot assign twice to immutable variable `_immutable_binding`

## Referencia y *Borrowing*

Para entender las Referencias y el *Borrowing* es bueno verlo con un ejemplo: 


In [ ]:
fn main() {
    let s1 = String::from("hello");

    let len = calculate_length(&s1);

    println!("The length of '{}' is {}.", s1, len);
}

fn calculate_length(s: &String) -> usize {
    s.len()
} // Here, s goes out of scope. But because it does not have ownership of what
  // it refers to, nothing happens.

Los ampersands son referencias y permiten referirse a algún valor sin tomar *ownership* de él. 

![image.png](attachment:image.png)/y


La sintaxis *&s1* nos permite crear una referencia que se refiere al valor de **s1** pero que no lo posee. Como no lo posee, el valor al que apunta no se descartará cuando la referencia salga del alcance (*scope*).

Qué pasa si traramos de **modificar** algo que pedimos prestado? 

In [32]:
fn main() {
    let s = String::from("hello");

    change(&s);
}

fn change(some_string: &String) {
    some_string.push_str(", world");
}

main()

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

Nos aparecerá un error como:  *some_string` is a `&` reference, so the data it refers to cannot be borrowed as mutable`*. 

> Hay que recordar que por defecto las variables son inmutables, al igual que las referencias. No podemos modificar algo a lo que tenemos referencia.


## Referencias mutables 

Podemos corregir el error en el código del Listado 4-6 con solo un pequeño ajuste:


In [33]:
fn main() {
    let mut s = String::from("hello");

    change(&mut s);
}

fn change(some_string: &mut String) {
    some_string.push_str(", world");
}

main()

()

Primero, tuvimos que **cambiar s** para ser *mut*. Luego tuvimos que crear una referencia mutable con *&mut s* y aceptar una referencia mutable con *some_string: & mut String*.

> Pero las referencias mutables tienen una gran restricción: solo puede tener una referencia mutable a un dato en particular en un ámbito particular. Este código fallará


In [34]:
let mut s = String::from("hello");

let r1 = &mut s;
let r2 = &mut s;

println!("{}, {}", r1, r2);

Error: `s` does not live long enough

Error: cannot borrow `s` as mutable more than once at a time

Error: cannot move out of `s` because it is borrowed

## Ejemplo de mutabilidad y Borrowing 

In [28]:
struct Book {
    // `&'static str` es una referencia a un string alocato en memoia de solo relctura 
    author: &'static str,
    title: &'static str,
    year: u32,
}

//  Funcion toma como referencia un Book
fn borrow_book(book: &Book) {
    println!("I immutably borrowed {} - {} edition", book.title, book.year);
}

// Funcion toma como referencia un Book mutable y cambia incrementa el year + 1
fn new_edition(book: &mut Book) {
    book.year = book.year + 1000 ;
    println!("I mutably borrowed {} - {} edition", book.title, book.year);
}

// Crea un libro inmutable `immutabook`
let immutabook = Book {
    // string literals have type `&'static str`
    author: "Douglas Hofstadter",
    title: "Gödel, Escher, Bach",
    year: 1979,
};
    
// Immutably borrow an immutable object
borrow_book(&immutabook);

// Create a mutable copy of `immutabook` and call it `mutabook`
let mut mutabook = immutabook;

// Immutably borrow a mutable object
borrow_book(&mutabook);
    
// Borrow a mutable object as mutable
new_edition(&mut mutabook);

//new_edition(&mut immutabook);

I immutably borrowed Gödel, Escher, Bach - 1979 edition
I immutably borrowed Gödel, Escher, Bach - 1979 edition
I mutably borrowed Gödel, Escher, Bach - 2979 edition
h 2979


## Ejemplo 

In [2]:
:dep threadpool = "1.7.1"
:dep num = "0.2.0"
:dep num_cpus = "1.11.1"
:dep image = "0.22.3"

extern crate threadpool;
extern crate num;
extern crate num_cpus;
extern crate image;

use std::error::Error;
use std::sync::mpsc::{channel, RecvError};
use threadpool::ThreadPool;
use num::complex::Complex;
use image::{ImageBuffer, Pixel, Rgb};

fn wavelength_to_rgb(wavelength: u32) -> Rgb<u8> {
     let wave = wavelength as f32;

     let (r, g, b) = match wavelength {
         380..=439 => ((440. - wave) / (440. - 380.), 0.0, 1.0),
         440..=489 => (0.0, (wave - 440.) / (490. - 440.), 1.0),
         490..=509 => (0.0, 1.0, (510. - wave) / (510. - 490.)),
         510..=579 => ((wave - 510.) / (580. - 510.), 1.0, 0.0),
         580..=644 => (1.0, (645. - wave) / (645. - 580.), 0.0),
         645..=780 => (1.0, 0.0, 0.0),
         _ => (0.0, 0.0, 0.0),
     };

     let factor = match wavelength {
         380..=419 => 0.3 + 0.7 * (wave - 380.) / (420. - 380.),
         701..=780 => 0.3 + 0.7 * (780. - wave) / (780. - 700.),
         _ => 1.0,
     };

     let (r, g, b) = (normalize(r, factor), normalize(g, factor), normalize(b, factor));
     Rgb::from_channels(r, g, b, 0)
 }

 // Maps Julia set distance estimation to intensity values
 fn julia(c: Complex<f32>, x: u32, y: u32, width: u32, height: u32, max_iter: u32) -> u32 {
     let width = width as f32;
     let height = height as f32;

     let mut z = Complex {
         // scale and translate the point to image coordinates
         re: 3.0 * (x as f32 - 0.5 * width) / width,
         im: 2.0 * (y as f32 - 0.5 * height) / height,
     };

     let mut i = 0;
     for t in 0..max_iter {
         if z.norm() >= 2.0 {
             break;
         }
         z = z * z + c;
         i = t;
     }
     i
 }

 // Normalizes color intensity values within RGB range
 fn normalize(color: f32, factor: f32) -> u8 {
     ((color * factor).powf(0.8) * 255.) as u8
 }

In [7]:
fn main() -> Result<(), Box<dyn Error>> {
    let (width, height) = (1920, 1080);
    let mut img = ImageBuffer::new(width, height);
    let iterations = 300;

    let c = Complex::new(-0.8,  0.20);

    let pool = ThreadPool::new(num_cpus::get());
    let (tx, rx) = channel();

    for y in 0..height {
        let tx1 = tx.clone();
        pool.execute(move || for x in 0..width/2 {
                         let i = julia(c, x, y, width, height, iterations);
                         let pixel = wavelength_to_rgb(380 + i * 400 / iterations);
                         tx1.send((x, y, pixel)).expect("Could not send data!");
                     });
        let tx2 = tx.clone();
        pool.execute(move || for x in width/2..width {
                         let i = julia(c, x, y, width, height, iterations);
                         let pixel = wavelength_to_rgb(380 + i * 400 / iterations);
                         tx2.send((x, y, pixel)).expect("Could not send data!");
                     });
    }

    for _ in 0..(width * height) {
        let (x, y, pixel) = rx.recv()?;
        img.put_pixel(x, y, pixel);
    }
    let _ = img.save("output.png")?;
    Ok(())
}

main()

Ok(())

![output.png](output.png)